**Cloning tensorflow models**

In [ ]:
import os
import pathlib
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

**Installing API**

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
import tensorflow as tf
import tarfile
import re

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

**Testing API**

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

**Created a shortcut of the dataset from [here](http://shuoyang1213.me/WIDERFACE/) in my personal Google Drive and unzipping them to session storage**

In [ ]:
%%bash
unzip /content/drive/MyDrive/Colab\ Notebooks/dataset/WIDER_train.zip
unzip /content/drive/MyDrive/Colab\ Notebooks/dataset/WIDER_val.zip
unzip /content/drive/MyDrive/Colab\ Notebooks/dataset/wider_face_split.zip

In [ ]:
%cd /content/WIDER_train/images/
%mv */* .
%rm -r */
%cd /content/WIDER_val/images/
%mv */* .
%rm -r */
%cd /content/

**Modifying gt files with correct paths**

In [ ]:
with open("/content/wider_face_split/wider_face_train_bbx_gt.txt", "r") as fp:
    lines = fp.readlines()
with open("/content/wider_face_split/wider_face_train_bbx_gt.txt", "wb") as fp:
    for line in lines:
      if(".jpg" in line):
        line = line.rsplit('/',1)[1]
      fp.write(line.encode())
with open("/content/wider_face_split/wider_face_val_bbx_gt.txt", "r") as fp:
    lines = fp.readlines()
with open("/content/wider_face_split/wider_face_val_bbx_gt.txt", "wb") as fp:
    for line in lines:
      if(".jpg" in line):
        line = line.rsplit('/',1)[1]
      fp.write(line.encode())

**Downloading necessary files**

In [ ]:
%%bash
wget https://raw.githubusercontent.com/GowthamGottimukkala/wobotai.assignment/main/convert.py
wget https://raw.githubusercontent.com/GowthamGottimukkala/wobotai.assignment/main/xml_to_csv.py
wget https://raw.githubusercontent.com/GowthamGottimukkala/wobotai.assignment/main/generate_tfrecord.py
wget https://raw.githubusercontent.com/GowthamGottimukkala/wobotai.assignment/main/generate_pbtxt.py

**Preparing data for the TF OD API**

*Converting from Widerface annotation format to xml and then to tfrecord*

In [ ]:
%%bash
python convert.py -ap /content/wider_face_split/wider_face_train_bbx_gt.txt -tp /content/XML_train -ip /content/WIDER_train/images
python xml_to_csv.py -xmlp /content/XML_train/ -tp /content/train.csv
python convert.py -ap /content/wider_face_split/wider_face_val_bbx_gt.txt -tp /content/XML_val -ip /content/WIDER_val/images
python xml_to_csv.py -xmlp /content/XML_val/ -tp /content/val.csv

In [ ]:
%%bash
python generate_pbtxt.py csv train.csv labelmap_pb.txt
python generate_tfrecord.py train.csv labelmap_pb.txt WIDER_train/images train.tfrecords
python generate_tfrecord.py val.csv labelmap_pb.txt WIDER_val/images val.tfrecords

**Downloading and extracting model**

In [ ]:
%mkdir /content/models/research/deploy/
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz -P /content/models/research/deploy
tarfile.open("/content/models/research/deploy/efficientdet_d0_coco17_tpu-32.tar.gz").extractall("/content/models/research/deploy/")

**Inializing parameters**

In [ ]:
num_classes = 1
num_steps = 1000
num_eval_steps = 200
batch_size = 4

**Downloading Configuration File**

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config -P /content/models/research/deploy

**Writing Custom Configuration file**

In [ ]:
fine_tune_checkpoint = '/content/models/research/deploy/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0'
%cd /content/models/research/deploy

with open('ssd_efficientdet_d0_512x512_coco17_tpu-8.config') as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format("/content/train.tfrecords"), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format("/content/val.tfrecords"), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format("/content/labelmap_pb.txt"), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

%cd /content

In [ ]:
!cat /content/models/research/deploy/pipeline_file.config

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

In [ ]:
!kill 930
%rm -rf /content/training

In [ ]:
%mkdir /content/training
%load_ext tensorboard
%tensorboard --logdir '/content/training/train'

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py --pipeline_config_path /content/models/research/deploy/pipeline_file.config --model_dir /content/training/ --alsologtostderr --num_train_steps={num_steps} --sample_1_of_n_eval_examples=1 --num_eval_steps={num_eval_steps}

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py --model_dir=/content/training/ --pipeline_config_path=/content/models/research/deploy/pipeline_file.config --checkpoint_dir=/content/training/